In [18]:
# Instalar a biblioteca pgmpy (se necessário)
%pip install pgmpy

# Importar bibliotecas
import numpy as np
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
import itertools

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pgmpy in c:\users\lucas\appdata\local\programs\python\python312\lib\site-packages (0.1.26)




[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
# Carregar os dados do Titanic
train_df = pd.read_csv('train.csv')

# Discretizar Age em faixas etárias
train_df['Age'] = pd.cut(train_df['Age'], bins=[0, 12, 18, 59, 120], labels=['criança', 'adolescente', 'adulto', 'idoso'])

# Discretizar Fare em categorias: baixo, médio, alto
train_df['Fare'] = pd.qcut(train_df['Fare'], q=3, labels=['baixo', 'médio', 'alto'])

# Derivar 'Family_Size' como a soma de 'SibSp' e 'Parch'
train_df['Family_Size'] = train_df['SibSp'] + train_df['Parch']

# Derivar 'Cabin_type' como uma variável categórica
# Aqui assumimos que, se a cabine começa com 'A', 'B', ou 'C', é alta, caso contrário é baixa.
train_df['Cabin_type'] = train_df['Cabin'].apply(lambda x: 'alta' if pd.notna(x) and x[0] in ['A', 'B', 'C'] else 'baixa')

# Adicionar a nova categoria 'n/a' às colunas categóricas antes de preencher os valores NaN
train_df['Age'] = train_df['Age'].cat.add_categories('n/a')
train_df['Embarked'] = train_df['Embarked'].astype('category').cat.add_categories('n/a')

# Visualizar as primeiras linhas para verificar a discretização e as variáveis derivadas
train_df[['Age', 'Fare', 'Family_Size', 'Cabin_type']].head()

train_df.fillna({
    'Age': 'n/a',
    'Embarked': 'n/a'
}, inplace=True)

In [20]:
epsilon = 1e-6

def calculate_conditional_probabilities(data, target, conditions):
    """
    Calcula as probabilidades condicionais de uma variável target dado um conjunto de condições,
    com base nos dados fornecidos.
    """
    if not conditions:
        # Caso não haja condições, só computa a proporção e normaliza
        counts = data[target].value_counts().sort_index()
        counts = counts.replace(0, epsilon)
        probabilities = counts / counts.sum()
        return pd.DataFrame([probabilities.values], columns=probabilities.index)

    # Obtém o conjunto dos valores possíveis para cada condição e para o target
    condition_values = [data[condition].unique() for condition in conditions]
    target_values = data[target].unique()

    # Gera todas as combinações possíveis de valores das condições
    all_combinations = list(itertools.product(*condition_values))
    index = pd.MultiIndex.from_tuples(all_combinations, names=conditions)

    # Agrupa os dados por condições e target
    grouped_data = data.groupby(conditions + [target]).size().unstack(fill_value=0)

    # Reindexa para incluir todas as combinações possíveis
    grouped_data = grouped_data.reindex(index=index, fill_value=0)

    # Reindexa de novo para incluir todas as categorias possíveis do target
    grouped_data = grouped_data.reindex(columns=target_values, fill_value=0)

    # Adiciona epsilon para evitar probabilidades nulas
    grouped_data = grouped_data.replace(0, epsilon)

    # Normaliza as probabilidades
    row_sums = grouped_data.sum(axis=1)
    conditional_probs = grouped_data.div(row_sums, axis=0)

    return conditional_probs

In [21]:
# Construir o modelo Bayesian Network com as variáveis derivadas
model = BayesianNetwork([
    ('Fare', 'Pclass'),
    ('Embarked', 'Pclass'),
    ('Pclass', 'Survived'),
    ('Sex', 'Survived'),
    ('Age', 'Survived'),
    ('Pclass', 'SibSp'),
    ('Pclass', 'Parch'),
    ('SibSp', 'Family_Size'),
    ('Parch', 'Family_Size'),
    ('Cabin_type', 'Survived'),
    ('Family_Size', 'Survived')
])


In [35]:
# CPD para Pclass, condicionado a Fare e Embarked
cpd_pclass_data = calculate_conditional_probabilities(train_df, 'Pclass', ['Fare', 'Embarked'])
print(cpd_pclass_data)
cpd_pclass = TabularCPD(
    variable='Pclass',
    variable_card=3,
    values=cpd_pclass_data.values.T.tolist(),
    evidence=['Fare', 'Embarked'],
    evidence_card=[3, 4]
)



Pclass                     3             1             2
Fare  Embarked                                          
baixo S         9.449541e-01  2.752294e-02  2.752294e-02
      C         9.999999e-01  2.777778e-08  2.777778e-08
      Q         1.000000e+00  1.851852e-08  1.851852e-08
      n/a       3.333333e-01  3.333333e-01  3.333333e-01
alto  S         2.435233e-01  5.958549e-01  1.606218e-01
      C         1.075269e-08  9.139785e-01  8.602150e-02
      Q         7.142856e-01  2.857142e-01  1.428571e-07
      n/a       4.999995e-07  9.999990e-01  4.999995e-07
médio S         4.291845e-01  2.575107e-02  5.450644e-01
      C         7.692307e-01  2.564102e-08  2.307692e-01
      Q         8.124999e-01  6.250000e-08  1.875000e-01
      n/a       3.333333e-01  3.333333e-01  3.333333e-01


C:\Users\lucas\AppData\Local\Temp\ipykernel_17276\146304179.py:25: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_data = data.groupby(conditions + [target]).size().unstack(fill_value=0)


In [23]:

# CPD para Family_size, condicionado a Survived
cpd_family_size_data = calculate_conditional_probabilities(train_df, 'Family_Size', ['Survived'])
cpd_family_size = TabularCPD(
    variable='Family_Size',
    variable_card=9,
    values=cpd_family_size_data.values.T.tolist(),
    evidence=['Survived'],
    evidence_card=[2]
)
model.add_cpds(cpd_family_size)


In [24]:
cpd_sex_data = calculate_conditional_probabilities(train_df, 'Sex', [])
cpd_sex = TabularCPD(variable='Sex', variable_card=2, values=cpd_sex_data.values.T.tolist())

In [25]:
cpd_age_data = calculate_conditional_probabilities(train_df, 'Age', [])
cpd_age = TabularCPD(variable='Age', variable_card=5, values=cpd_age_data.values.T.tolist())

In [26]:
cpd_fare_data = calculate_conditional_probabilities(train_df, 'Fare', [])
cpd_fare = TabularCPD(variable='Fare', variable_card=3, values=cpd_fare_data.values.T.tolist())

In [27]:
cpd_embarked_data = calculate_conditional_probabilities(train_df, 'Embarked', [])
cpd_embarked = TabularCPD(variable='Embarked', variable_card=4, values=cpd_embarked_data.values.T.tolist())

In [28]:
# CPD para SibSp baseado em Pclass
cpd_sibsp_data = calculate_conditional_probabilities(train_df, 'SibSp', ['Pclass'])
cpd_sibsp = TabularCPD(
    variable='SibSp',
    variable_card=7,
    values=cpd_sibsp_data.values.T.tolist(),
    evidence=['Pclass'],
    evidence_card=[3]
)


In [29]:
# CPD para Parch baseado em Pclass
cpd_parch_data = calculate_conditional_probabilities(train_df, 'Parch', ['Pclass'])
cpd_parch = TabularCPD(
    variable='Parch',
    variable_card=7,
    values=cpd_parch_data.values.T.tolist(),
    evidence=['Pclass'],
    evidence_card=[3]
)

In [30]:
# CPD para Family_Size, baseado em SibSp e Parch
cpd_family_size_data = calculate_conditional_probabilities(train_df, 'Family_Size', ['SibSp', 'Parch'])
cpd_family_size = TabularCPD(
    variable='Family_Size',
    variable_card=9,
    values=cpd_family_size_data.values.T.tolist(),
    evidence=['SibSp', 'Parch'],
    evidence_card=[7, 7]
)

In [31]:
cpd_cabin_type_data = calculate_conditional_probabilities(train_df, 'Cabin_type', [])
cpd_cabin_type = TabularCPD(variable='Cabin_type', variable_card=2, values=cpd_cabin_type_data.values.T.tolist())

In [32]:
# CPD para Survived, atualizado com Cabin_type como evidência adicional
cpd_survived_data = calculate_conditional_probabilities(train_df, 'Survived', ['Pclass', 'Sex', 'Age', 'Cabin_type', 'Family_Size'])
cpd_survived = TabularCPD(
    variable='Survived',
    variable_card=2,
    values=cpd_survived_data.values.T.tolist(),
    evidence=['Pclass', 'Sex', 'Age', 'Cabin_type', 'Family_Size'],
    evidence_card=[3, 2, 5, 2, 9]
)


C:\Users\lucas\AppData\Local\Temp\ipykernel_17276\146304179.py:25: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_data = data.groupby(conditions + [target]).size().unstack(fill_value=0)


In [33]:
# Adicionar todos os CPDs ao modelo, incluindo os novos
model.add_cpds(cpd_pclass, cpd_sex, cpd_age, cpd_fare, cpd_embarked, cpd_sibsp, cpd_parch, cpd_family_size, cpd_cabin_type, cpd_survived)

# Validar o modelo
model_valid = model.check_model()
print("Modelo válido:", model_valid)

Modelo válido: True


In [34]:
# Configurar o inferenciador da rede
inference = VariableElimination(model)

print("Probabilidade de sobrevivência dado que o passageiro está nas classes:")
classes = train_df['Pclass'].unique()
for i,c in enumerate(classes):
    prob_survived_given_class = inference.query(variables=['Survived'], evidence={'Pclass': i})
    print(f"Classe {c}:")
    print(prob_survived_given_class)

print("Probabilidade de sobrevivência dado sexo")
sexes = train_df['Sex'].unique()
for i,s in enumerate(sexes):
    prob_survived_given_class = inference.query(variables=['Survived'], evidence={'Sex': i})
    print(f"Sexo {s}:")
    print(prob_survived_given_class)

print("Probabilidade do passageiro ter uma família grande dado que ele sobreviveu")
prob_family_size_given_survived = inference.query(variables=['Family_Size'], evidence={'Survived': 1})
print(prob_family_size_given_survived)


Probabilidade de sobrevivência dado que o passageiro está nas classes:
Classe 3:
+-------------+-----------------+
| Survived    |   phi(Survived) |
+=============+=================+
| Survived(0) |          0.5082 |
+-------------+-----------------+
| Survived(1) |          0.4918 |
+-------------+-----------------+
Classe 1:
+-------------+-----------------+
| Survived    |   phi(Survived) |
+=============+=================+
| Survived(0) |          0.4861 |
+-------------+-----------------+
| Survived(1) |          0.5139 |
+-------------+-----------------+
Classe 2:
+-------------+-----------------+
| Survived    |   phi(Survived) |
+=============+=================+
| Survived(0) |          0.4853 |
+-------------+-----------------+
| Survived(1) |          0.5147 |
+-------------+-----------------+
Probabilidade de sobrevivência dado sexo
Sexo male:
+-------------+-----------------+
| Survived    |   phi(Survived) |
+=============+=================+
| Survived(0) |          0.5057